<a href="https://colab.research.google.com/github/KhannaShivang/Market_predictor/blob/main/marketprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf

In [ ]:
sp500 = yf.Ticker("^GSPC")

In [ ]:
sp500 = sp500.history(period="max")
sp500.index

In [ ]:
sp500.plot.line(y="Close",use_index=True)

In [ ]:
del sp500["Dividends"]
del sp500["Stock Splits"]

In [ ]:
sp500["Tomorrow"] = sp500["Close"].shift(-1)

In [ ]:
sp500["Target"]=(sp500["Tomorrow"] > sp500["Close"]).astype(int)
sp500

In [ ]:
sp500 = sp500.loc["1990-01-01":].copy()
sp500

,Open,High,Low,Close,Volume,Tomorrow,Target
Date,,,,,,,
1990-01-02 00:00:00-05:00,353.399994,359.690002,351.980011,359.690002,162070000,358.760010,0
1990-01-03 00:00:00-05:00,359.690002,360.589996,357.890015,358.760010,192330000,355.670013,0
1990-01-04 00:00:00-05:00,358.760010,358.760010,352.890015,355.670013,177000000,352.200012,0
1990-01-05 00:00:00-05:00,355.670013,355.670013,351.350006,352.200012,158530000,353.790009,1
1990-01-08 00:00:00-05:00,352.200012,354.239990,350.540009,353.790009,140110000,349.619995,0
...,...,...,...,...,...,...,...
2024-08-26 00:00:00-04:00,5639.660156,5651.620117,5602.339844,5616.839844,2938570000,5625.799805,1
2024-08-27 00:00:00-04:00,5602.890137,5631.180176,5593.479980,5625.799805,2798990000,5592.180176,0
2024-08-28 00:00:00-04:00,5624.509766,5627.029785,5560.950195,5592.180176,3053450000,5591.959961,0


In [ ]:
#RandomForestClasifier works by traininng a bunch of dicision trees with randomized parametersand the averaging the result
#because of this process RandomForest are resistent to overfit also run prty quickly and can pickup non linear tendencies in data
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators =  100, min_samples_split = 100, random_state=1)
#n_estimaters: number of individual decision tree you want to train high it is hight the accurecy up to limit
#min_sample_size: protect againg overfiting the higher we set it the less accurate the model will be but will prevent mre overfiting
#random_state: A random forest has some randomasation built in setting random state means that if we run the same mpdel twice the random numbers generated will be in a pridictable sequence runnunt twice gice same result
train = sp500.iloc[:-100]
test = sp500.iloc[-100:]
predictors =["Close", "Volume", "Open", "High", "Low"]
model.fit(train[predictors],train["Target"])

RandomForestClassifier(min_samples_split=100, random_state=1)

In [ ]:
from sklearn.metrics import precision_score

preds = model.predict(test[predictors])
preds

In [ ]:
import pandas as pd
preds=pd.Series(preds, index=test.index)
precision_score(test["Target"], preds)

0.5882352941176471

In [ ]:
combine = pd.concat([test["Target"], preds],axis = 1)
combine.plot()

In [ ]:
def predict(train, test, predictors, model):
  model.fit(train[predictors], train["Target"])
  preds = model.predict(test[predictors])
  preds = pd.Series(preds, index=test.index, name="Predictions")
  combined = pd.concat([test["Target"], preds], axis = 1)
  return combined

In [ ]:
def backtest(data, model, predictors, start= 2500, step=250):
  all_prediction = []
  for i in range(start, data.shape[0], step):
    train = data.iloc[0:i].copy()
    test = data.iloc[i:(i+step)].copy()
    predictions = predict(train, test, predictors, model)
    all_prediction.append(predictions)
  return pd.concat(all_prediction)

In [ ]:
sp500

In [ ]:
predictions = backtest(sp500, model, predictors)
predictions["Predictions"].value_counts()

In [ ]:
precision_score(predictions["Target"], predictions["Predictions"])

In [ ]:
predictions["Target"].value_counts()/predictions.shape[0]

In [ ]:
horizons = [2,5,60,250,1000]
new_predictors = []
for horizon in horizons:
  rolling_averages = sp500.rolling(horizon).mean()
  ratio_column = f"Close_Ratio_{horizon}"
  sp500[ratio_column] = sp500["Close"]/rolling_averages["Close"]

  trend_column = f"Trend_{horizon}"
  sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]

  new_predictors += [ratio_column, trend_column]
sp500 = sp500.dropna(subset=sp500.columns[sp500.columns != "Tomorrow"])
sp500

In [ ]:
model = RandomForestClassifier(n_estimators = 200, min_samples_split=50, random_state=1)

In [ ]:
sp500

In [ ]:
def predict(train, test, predictors, model):
  model.fit(train[predictors], train["Target"])
  preds = model.predict_proba(test[predictors])[:,1]
  preds[preds >= 0.6] =1
  preds[preds <0.6] = 0
  preds = pd.Series(preds, index=test.index, name="Predictions")
  combined = pd.concat([test["Target"], preds], axis = 1)
  combined
  return combined

In [ ]:
predictions = backtest(sp500, model, new_predictors)

In [ ]:
predictions

In [ ]:
predictions["Predictions"].value_counts()

In [ ]:
precision_score(predictions["Target"], predictions["Predictions"])


In [ ]:
predictions["Target"].value_counts() / predictions.shape[0]

In [ ]:
predictions